In [1]:
from mindquantum.core.gates import RX, XX
from mindquantum.core.circuit import Circuit

circ = Circuit()
circ += XX('a').on([0, 1], 2)

print(circ)

q0: ──XX(a)──
        │
q1: ──XX(a)──
        │
q2: ────●────


$$
\begin{align*}
R_{ij}^{\sigma_a \sigma_b} & = 
e^{-i\theta \sigma_a^i \otimes \sigma_b^j} \\
& = \cos(\theta) I^i\otimes I^j - i \sin(\theta) \sigma_a^i\otimes \sigma_b^j
\end{align*}
$$

In [2]:
from mindquantum.core.operators import QubitOperator
from mindquantum.core.operators import TimeEvolution
from mindquantum.core.circuit import apply

# X_i Y_j(alpha)

a = QubitOperator("X0 Y1", {"alpha": 2})
u = TimeEvolution(a).circuit
circ = apply(u, [2, 5])
print(circ)

q2: ─────H───────●───────────────────●───────H──────
                 │                   │
q5: ──RX(π/2)────X────RZ(4*alpha)────X────RX(7π/2)──


In [3]:
def R_i_j_a_b(i: int, j: int, sigma1: str, sigma2: str, pr):
    from mindquantum.core.operators import QubitOperator, TimeEvolution
    from mindquantum.core.circuit import apply

    if sigma1 not in "XYZ" or sigma2 not in "XYZ":
        raise ValueError
    op = QubitOperator(f"{sigma1}0 {sigma2}1", pr)
    u = TimeEvolution(op).circuit
    circ = apply(u, [i, j])
    return circ

In [4]:
def r_double_gate(i: int, j: int, name: str):
    sign = 1
    if name[0] == "-":
        sign = -1
        name = name[1:]
    return R_i_j_a_b(i, j, name[0], name[1], {name : sign})

## Bogoliubov Transformation

![Bob](./imag/Bob.png)

$$
\begin{align}
R_{ij}^{-YX(k)} & = 
e^{-i\theta_{ij}^{AA(k)}\sigma_y^i \otimes \sigma_x^j} \\

R_{ij}^{XY(k)} & = 
e^{i\theta_{ij}^{BB(k)}\sigma_x^i \otimes \sigma_y^j} \\

R_{ij}^{-YY(k)} & = 
e^{-i\theta_{ij}^{AB(k)}\sigma_y^i \otimes \sigma_y^j} \\

R_{ij}^{XX(k)} & = 
e^{i\theta_{ij}^{BA(k)}\sigma_x^i \otimes \sigma_x^j} \\
\end{align}
$$


In [5]:
def G_i_j_k(i: int, j: int, k: int):
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    circ += r_double_gate(i, j, f"-YX({k})")
    circ += r_double_gate(i, j, f"XY({k})")
    circ += r_double_gate(i, j, f"-YY({k})")
    circ += r_double_gate(i, j, f"XX({k})")
    return circ

In [6]:
def U_MG_k(n_qubits: int, k: int):
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for i in range(n_qubits // 2):
        circ += G_i_j_k(2 * i, 2 * i + 1, k)
    for i in range(n_qubits // 2 - 1):
        circ += G_i_j_k(2 * i + 1, 2 * i + 2, k)
    return circ

In [7]:
def U_MG_NN(n_qubits: int):
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for k in range(n_qubits // 2):
        circ += U_MG_k(n_qubits, k)
    return circ

In [8]:
def U_Bog(n_qubits: int):
    from mindquantum.core.gates import RZ
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for i in range(n_qubits):
        circ += RZ(f"theta_{i}^AB").on(i)
    circ += U_MG_NN(n_qubits)
    return circ

## Low-depth Circuit Ansatz

![LDCA](./imag/LDCA.png)


$$
\begin{align*}
R_{ij}^{-YX(k,l)} & = 
e^{-i\theta_{ij}^{-YX(k,l)}\sigma_y^i\otimes \sigma_x^j} \\

R_{ij}^{XY(k,l)} & = 
e^{i\theta_{ij}^{XY(k,l)}\sigma_x^i\otimes \sigma_y^j} \\

R_{ij}^{ZZ(k,l)} & = 
e^{i\theta_{ij}^{ZZ(k,l)}\sigma_z^i\otimes \sigma_z^j} \\

R_{ij}^{-YY(k,l)} & = 
e^{-i\theta_{ij}^{-YY(k,l)}\sigma_y^i\otimes \sigma_y^j} \\

R_{ij}^{XX(k,l)} & = 
e^{i\theta_{ij}^{XX(k,l)}\sigma_x^i\otimes \sigma_x^j} \\
\end{align*}
$$

In [9]:
def K_i_j_k_l(i: int, j: int, k: int, l: int):
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    circ += r_double_gate(i, j, f"-YX({k},{l})")
    circ += r_double_gate(i, j, f"XY({k},{l})")
    circ += r_double_gate(i, j, f"ZZ({k},{l})")
    circ += r_double_gate(i, j, f"-YY({k},{l})")
    circ += r_double_gate(i, j, f"XX({k},{l})")
    return circ

In [10]:
def U_VarMG_k_l(n_qubits: int, k: int, l: int):
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for i in range(n_qubits // 2):
        circ += K_i_j_k_l(2 * i, 2 * i + 1, k, l)
    for i in range(n_qubits // 2 - 1):
        circ += K_i_j_k_l(2 * i + 1, 2 * i + 2, k, l)
    return circ

In [11]:
def U_VarMG_l(n_qubits: int, l: int):
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for k in range(n_qubits // 2):
        circ += U_VarMG_k_l(n_qubits, k, l)
    return circ

In [12]:
def U_VarMG(n_qubits: int, L: int):
    from mindquantum.core.gates import RZ
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for i in range(n_qubits):
        circ += RZ(f"theta_{i}^Z").on(i)
    for l in range(L):
        circ += U_VarMG_l(n_qubits, l)
    return circ

In [13]:
def LDCA(n_qubits: int, L: int):
    from mindquantum.core.gates import X 
    from mindquantum.core.circuit import Circuit
    circ = Circuit()
    for i in range(n_qubits):
        circ += X.on(i)
    circ += U_VarMG(n_qubits, L)
    circ += U_Bog(n_qubits).hermitian()
    return circ

In [14]:
circ = LDCA(8, 2)
circ.summary()

============================================================Circuit Summary============================================================
|Total number of gates  : 4112.                                                                                                       |
|Parameter gates        : 408.                                                                                                        |
|with 72 parameters are : theta_0^Z, theta_1^Z, theta_2^Z, theta_3^Z, theta_4^Z, theta_5^Z, theta_6^Z, theta_7^Z, YX(0,0), XY(0,0)... |
|Number qubit of circuit: 8                                                                                                           |


In [15]:
circ = LDCA(4, 2)
circ.summary()

========================================================Circuit Summary========================================================
|Total number of gates  : 888.                                                                                                |
|Parameter gates        : 92.                                                                                                 |
|with 36 parameters are : theta_0^Z, theta_1^Z, theta_2^Z, theta_3^Z, YX(0,0), XY(0,0), ZZ(0,0), YY(0,0), XX(0,0), YX(1,0)... |
|Number qubit of circuit: 4                                                                                                   |
